<a href="https://colab.research.google.com/github/kjedrzejczak/Wprowadzenie_do_sieci_neuronowych-Tensorflow_2.0-Keras/blob/main/Przeuczenie_niedouczenia_I_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle


import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets.imdb import get_word_index
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


sns.set()

In [2]:
NUM_WORDS = 10000   # 10000 najszęstrzych słów
INDEX_FROM = 3

(train_data, train_lables), (test_data, test_lables) = imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)

In [3]:
print(train_data.shape)
print(train_lables.shape)
print(test_data.shape)
print(test_lables.shape)

(25000,)
(25000,)
(25000,)
(25000,)


In [4]:
train_lables[0]

1

In [5]:
print(train_data[0])  # częstośc występowania słowa w recenzjach np. 14 - słowo wystąpiło na  14 miejscu w recenzjach

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [6]:
np.unique(train_lables)

array([0, 1])

In [7]:
# tłumaczenie słów na tekst przy pomocy funkcji get_word_index()
word_to_idx = get_word_index()
word_to_idx = {k:(v + INDEX_FROM) for k, v in word_to_idx.items()}
word_to_idx['<PAD>'] = 0
word_to_idx['<start>'] = 1
word_to_idx['<UKN>'] = 2
word_to_idx['<UNUSED'] = 3
idx_to_word = {v: k for k, v in word_to_idx.items()}
list(idx_to_word.items())[:10]
print(' '.join(idx_to_word[idx] for idx in train_data[0]))

<start> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UKN> is an amazing actor and now the same being director <UKN> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UKN> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UKN> to the two little boy's that played the <UKN> of norman and paul they were just brilliant children are often left out of the <UKN> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

In [8]:
train_lables[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [9]:
def multi_hot_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0
    return results

train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)
train_data.shape

(25000, 10000)

In [10]:
test_data.shape

(25000, 10000)

# Budowanie modelu

In [11]:
baseline_model = Sequential()
baseline_model.add(Dense(units=16, activation='relu', input_shape=(NUM_WORDS,)))   # NUM_WORDS = 10_000
baseline_model.add(Dense(units=16, activation='relu'))
baseline_model.add(Dense(units=1, activation='sigmoid'))  # blasifikacja binarna

baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

baseline_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                160016    
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160305 (626.19 KB)
Trainable params: 160305 (626.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
baseline_history = baseline_model.fit(train_data, train_lables, epochs=20, batch_size=512, validation_data=(test_data, test_lables), verbose=2)

Epoch 1/20
49/49 - 6s - loss: 0.4753 - accuracy: 0.8150 - binary_crossentropy: 0.4753 - val_loss: 0.3286 - val_accuracy: 0.8764 - val_binary_crossentropy: 0.3286 - 6s/epoch - 130ms/step
Epoch 2/20
49/49 - 3s - loss: 0.2427 - accuracy: 0.9120 - binary_crossentropy: 0.2427 - val_loss: 0.2845 - val_accuracy: 0.8862 - val_binary_crossentropy: 0.2845 - 3s/epoch - 57ms/step
Epoch 3/20
49/49 - 2s - loss: 0.1755 - accuracy: 0.9380 - binary_crossentropy: 0.1755 - val_loss: 0.2949 - val_accuracy: 0.8837 - val_binary_crossentropy: 0.2949 - 2s/epoch - 43ms/step
Epoch 4/20
49/49 - 2s - loss: 0.1381 - accuracy: 0.9534 - binary_crossentropy: 0.1381 - val_loss: 0.3220 - val_accuracy: 0.8782 - val_binary_crossentropy: 0.3220 - 2s/epoch - 50ms/step
Epoch 5/20
49/49 - 3s - loss: 0.1131 - accuracy: 0.9625 - binary_crossentropy: 0.1131 - val_loss: 0.3605 - val_accuracy: 0.8718 - val_binary_crossentropy: 0.3605 - 3s/epoch - 52ms/step
Epoch 6/20
49/49 - 3s - loss: 0.0932 - accuracy: 0.9691 - binary_crossentr

# Budowa mniejszego modelu

In [13]:
smaller_model = Sequential()
smaller_model.add(Dense(units=4, activation='relu', input_shape=(NUM_WORDS,)))   # NUM_WORDS = 10_000
smaller_model.add(Dense(units=4, activation='relu'))
smaller_model.add(Dense(units=1, activation='sigmoid'))

smaller_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

smaller_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 4)                 40004     
                                                                 
 dense_4 (Dense)             (None, 4)                 20        
                                                                 
 dense_5 (Dense)             (None, 1)                 5         
                                                                 
Total params: 40029 (156.36 KB)
Trainable params: 40029 (156.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
smaller_history = smaller_model.fit(train_data, train_lables, epochs=20, batch_size=512, validation_data=(test_data, test_lables), verbose=2)

Epoch 1/20
49/49 - 6s - loss: 0.6170 - accuracy: 0.6516 - binary_crossentropy: 0.6170 - val_loss: 0.5502 - val_accuracy: 0.7864 - val_binary_crossentropy: 0.5502 - 6s/epoch - 127ms/step
Epoch 2/20
49/49 - 2s - loss: 0.4905 - accuracy: 0.8474 - binary_crossentropy: 0.4905 - val_loss: 0.4761 - val_accuracy: 0.8722 - val_binary_crossentropy: 0.4761 - 2s/epoch - 42ms/step
Epoch 3/20
49/49 - 2s - loss: 0.4133 - accuracy: 0.9007 - binary_crossentropy: 0.4133 - val_loss: 0.4230 - val_accuracy: 0.8806 - val_binary_crossentropy: 0.4230 - 2s/epoch - 36ms/step
Epoch 4/20
49/49 - 3s - loss: 0.3489 - accuracy: 0.9264 - binary_crossentropy: 0.3489 - val_loss: 0.3830 - val_accuracy: 0.8804 - val_binary_crossentropy: 0.3830 - 3s/epoch - 58ms/step
Epoch 5/20
49/49 - 3s - loss: 0.2781 - accuracy: 0.9411 - binary_crossentropy: 0.2781 - val_loss: 0.3184 - val_accuracy: 0.8839 - val_binary_crossentropy: 0.3184 - 3s/epoch - 60ms/step
Epoch 6/20
49/49 - 3s - loss: 0.1863 - accuracy: 0.9505 - binary_crossentr

# Budowa większego modelu

In [15]:
bigger_model = Sequential()
bigger_model.add(Dense(units=256, activation='relu', input_shape=(NUM_WORDS,)))   # NUM_WORDS = 10_000
bigger_model.add(Dense(units=256, activation='relu'))
bigger_model.add(Dense(units=1, activation='sigmoid'))

bigger_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

bigger_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 256)               2560256   
                                                                 
 dense_7 (Dense)             (None, 256)               65792     
                                                                 
 dense_8 (Dense)             (None, 1)                 257       
                                                                 
Total params: 2626305 (10.02 MB)
Trainable params: 2626305 (10.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
bigger_history = bigger_model.fit(train_data, train_lables, epochs=20, batch_size=512, validation_data=(test_data, test_lables), verbose=2)

Epoch 1/20
49/49 - 14s - loss: 0.3574 - accuracy: 0.8458 - binary_crossentropy: 0.3574 - val_loss: 0.2931 - val_accuracy: 0.8801 - val_binary_crossentropy: 0.2931 - 14s/epoch - 285ms/step
Epoch 2/20
49/49 - 16s - loss: 0.1633 - accuracy: 0.9417 - binary_crossentropy: 0.1633 - val_loss: 0.3212 - val_accuracy: 0.8740 - val_binary_crossentropy: 0.3212 - 16s/epoch - 332ms/step
Epoch 3/20
49/49 - 10s - loss: 0.0832 - accuracy: 0.9733 - binary_crossentropy: 0.0832 - val_loss: 0.3950 - val_accuracy: 0.8717 - val_binary_crossentropy: 0.3950 - 10s/epoch - 208ms/step
Epoch 4/20
49/49 - 10s - loss: 0.0243 - accuracy: 0.9947 - binary_crossentropy: 0.0243 - val_loss: 0.5109 - val_accuracy: 0.8641 - val_binary_crossentropy: 0.5109 - 10s/epoch - 210ms/step
Epoch 5/20
49/49 - 12s - loss: 0.0043 - accuracy: 0.9998 - binary_crossentropy: 0.0043 - val_loss: 0.6135 - val_accuracy: 0.8644 - val_binary_crossentropy: 0.6135 - 12s/epoch - 237ms/step
Epoch 6/20
49/49 - 9s - loss: 8.6135e-04 - accuracy: 1.0000 

In [17]:
hist = pd.DataFrame(baseline_history.history)
hist['epoch'] = baseline_history.epoch
hist.head()

,loss,accuracy,binary_crossentropy,val_loss,val_accuracy,val_binary_crossentropy,epoch
0,0.475250,0.81500,0.475250,0.328570,0.87644,0.328570,0
1,0.242729,0.91204,0.242729,0.284450,0.88616,0.284450,1
2,0.175484,0.93800,0.175484,0.294915,0.88368,0.294915,2
3,0.138140,0.95340,0.138140,0.322016,0.87820,0.322016,3
4,0.113062,0.96248,0.113062,0.360460,0.87184,0.360460,4


In [18]:
import plotly.graph_objects as go

fig = go.Figure()
for name, history in zip(['smaller', 'baseline', 'bigger'], [smaller_history, baseline_history, bigger_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

## val_binary_crossentropy - najważniejszy - trzeba znaleśc minimum
Najlepszy jest model najmniejszy i należy go ograniczyć do tylko 5 epok !!!!